## General information

이 내용은 안드레이 룩야넨코의 노트북을 한글화한 것 입니다.
이 커널에서는 IEEE Fraud Detection competition을 다룹니다.

IEEE-CIS는 다양한 분야의 AI 및 머신러닝 부분에서 일하고 있는데요	
이는 deep neural networks, fuzzy systems, evolutionary computation 및 swarm intelligence를 포함합니다
이제 세계 굴지의 지불 시스템 회사인 Vesta와 협력하여, 카드 사기 방지를 할 수 있는 최적의 솔루션을 찾아서 수 많은 데이터 사이언티스트에게 이에 도전하라고 했습니다

이 챌린지는 binary classification problem으로 이러한 과제가 경험할 수 밖에 없는 data의 심한 imbalance를 보여 줍니다.
우리는 여기서 데이터를 탐구하고 소중한 인사이트를 파악하여 많은 feature engineering을 통하여 성능이 높은 모델링을 하려고 합니다

![](https://cis.ieee.org/images/files/slideshow/abstract01.jpg)


Importing libraries

In [ ]:
# importing libraries
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from IPython.display import display
pd.options.display.precision = 15
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000
pd.options.display.max_colwidth = 1000

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime
from numba import jit
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
import lightgbm as lgb

%load_ext autoreload
%autoreload 2


In [ ]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

seed = 42
seed_everything(seed)

## Data overview

데이터를 우선 로딩하고 보도록 하겠습니다.

In [ ]:
folder_path = '../input/'
train_identity = pd.read_csv(f'{folder_path}train_identity.csv')
train_transaction = pd.read_csv(f'{folder_path}train_transaction.csv')
test_identity = pd.read_csv(f'{folder_path}test_identity.csv')
test_transaction = pd.read_csv(f'{folder_path}test_transaction.csv')
sub = pd.read_csv(f'{folder_path}sample_submission.csv')
# let's combine the data and work with the whole dataset
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

        구글 콜랩을 사용하시면 데이터를 PC에 다운로드 하신 후 그 파일들을 아래 코드를 활용하여 다시 업로드 함으로 import 하세요 

        from google.colab import files
        uploaded = files.upload()

        이 다음에 파일 선택하고 (미리 다운로드 받아 놓으셔야 해요) 아래 코드로 변환 시키면 됩니다.

        그런데 콜래벵서 업로드 하면 시간이 엄청 걸립니다. (콜랩에서는 고용량 RAM 체크하고 하셔야 합니다)
        
        이어서 아래 실행

        import io
        train_identity = pd.read_csv(io.BytesIO(uploaded['train_identity.csv']))
        train_transaction = pd.read_csv(io.BytesIO(uploaded['train_transaction.csv']))
        test_identity = pd.read_csv(io.BytesIO(uploaded['test_identity.csv']))
        test_transaction = pd.read_csv(io.BytesIO(uploaded['test_transaction.csv']))
        sub = pd.read_csv(io.BytesIO(uploaded['sample_submission.csv']))

In [ ]:
train_identity.head()

우리는 열이 많은 꽤 큰 데이터셋 두 개를 볼 수 있습니다. Train과 test data는 비슷한 길이의 행을 가지고 있습니다.

In [ ]:
train_identity.shape

In [ ]:
train_transaction.head()

In [ ]:
train_transaction.shape

In [ ]:
test_identity.head()

In [ ]:
test_transaction.head()

데이터는 두 개로 나누어져 있는 것 같습니다: identity와 transaction.

아무래도 identity는 어떤 카드인가에 초점이 맞추어 지겠고 id 30또는 31을 보면 안드로이드나 크롬 같은 단어도 나옵니다?

Identity 40개 가량의 열이 있고 열의 내용을 눈치채게 할 이름은 없습니다.

하지만 사용자의 버전을 볼 수 있죠: OS, browser version, resolution, type 및 기타 등 등 

Transaction data는 거의 400 columns가 있습니다!

주목 할만한 열들은 다음과 같습니다:

card information

transaction date & amount

e-mail

이제 데이터를 합쳐 봅니다.

In [ ]:
# 어떤 이유에선지 테스트 컬럼에서 언더바 대신 하이픈으로 쓴 것이 있어서 좀 바꿉니다.
test_identity.columns = [col.replace('-', '_') for col in test_identity.columns]

# merge method를 사용하여 트레인과 테스트의 트랜잭션(거래)와 아이덴티티(신분)을 파일을 합합니다. 
# 여기서 on은 그 것을 인덱스로 합치라는 것이고 left나 right는 왼쪽 것 기준으로 또는 오른쪽 것 기준으로 파일을 정열하는데...여기서 말하는 기준은 데이터 값이 있는 것입니다.
# 다시 말해 left로 되어 있는데 왼쪽의 transaction의 해당 행이 빈 값이면 identity에 값이 있어도 그 행은 보이지 않게되고, 반대로 right라면 그 행은 보이게 됩니다.

train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

그리고 기존 데이터는 무거우니 지웁니다.

In [ ]:
del train_transaction, train_identity, test_transaction, test_identity

트레인 데이터에는 C1부터 C15까지 빈 값이 없으나 테스트 데이터에서는 빈값이 있어서 이들의 빈 값을 0으로 채웁니다.

아래에서 f는 f-string이라 불리는 것으로 이를 통해 다양한 표현식을 사용할 수 있다.

예를 들어 python2에서 나오는 %나 python3에서 나오는 str.format과 비교하였을 때 훨씬 간단하게 표현할 수 있다 

In [ ]:
test[[f'C{i}' for i in range(1,15)]] = test[[f'C{i}' for i in range(1,15)]].fillna(0)

이어서 얼마나 많은 램을 쓰는지 한 번 보기로 하자

In [ ]:
end_mem = train.memory_usage().sum() / 1024 ** 2 + test.memory_usage().sum() / 1024 ** 2
print(f'Mem. usage {end_mem:5.2f} Mb')

이는 상당히 많이 쓴 것으로 RAM사용량을 줄이는 메소드가 있으므로 사용해 보도록 합니다

정보는 그대로 주고 데이터 타입을 줄이므로 이를 달성합니다.

아시다시피 integer가 float보다 메모리를 덜 사용하고 integer8이 integer16보다 메모리를 덜 사용합니다.

일단 낮출 수 잇는 만큼 낮추어 보겠습니다.

In [ ]:
def reduce_mem_usage(memory_df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = memory_df.memory_usage().sum() / 1024 ** 2
    for col in memory_df.columns:
        col_type = memory_df[col].dtypes
        if col_type in numerics:
            c_min = memory_df[col].min()
            c_max = memory_df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    memory_df[col] = memory_df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    memory_df[col] = memory_df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    memory_df[col] = memory_df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    memory_df[col] = memory_df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    memory_df[col] = memory_df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    memory_df[col] = memory_df[col].astype(np.float32)
                else:
                    memory_df[col] = memory_df[col].astype(np.float64)
    end_mem = memory_df.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
            start_mem - end_mem) / start_mem))
    return memory_df

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

메모리 사용도가 2/3가 줄어 1/3 수준으로 줄어들었습니다.

## Data Exploration

이제 identity information을 파악해 봅니다.

id_01 - id_11는 continuous variables입니다.

id_12 - id_38는 categorical variable이고 마지막 두 개의 열은 확실히 범주 항목입니다.

In [ ]:
train.head()

In [ ]:
plt.hist(train['id_01'], bins=77);
plt.title('Distribution of id_01 variable');

`id_01` 은 아주 흥미로운 분포를 가지고 있습니다. 77개의 유니크한  양수가 아닌 수로 0에서 비대칭이 일어나는 분포입니다.

In [ ]:
train['id_03'].value_counts(dropna=False, normalize=True).head()

`id_03` 은 88%가 빈 값이며 98%가 빈값이거나 0입니다.

In [ ]:
train['id_11'].value_counts(dropna=False, normalize=True).head()

`id_11` 의 값 중 22%는 100이며 76%는 빈 값입니다. 좀 이상합니다.

In [ ]:
plt.hist(train['id_07']);
plt.title('Distribution of id_07 variable');

일부 항목은 normalized 된 것 같습니다. 모든 항목을 normalize하려면 이미 normalized 된 것은 분리하여야 할 것입니다. 

이제 transaction data를 살펴 보겠습니다.

In [ ]:
plt.hist(train['TransactionDT'], label='train');
plt.hist(test['TransactionDT'], label='test');
plt.legend();
plt.title('Distribution of transactiond dates');

여기서 중요점이 발견 됩니다.

트레인 및 테스트 트랜잭션 날짜가 겹치지 않는 것 같으므로 유효성 검사를 위해 시간 기반 분할을 사용하는 것이 현명할 것입니다.

대회 설명에서도 다음과 같은 내용이 있었습니다. "TransactionDT" 항목은 주어진 참조 날짜 / 시간 (실제 타임 스탬프 아님)의 타임 델타입니다.

시작일이 2017-11-30이라는 것을 깨달았습니다. 이 정보는 나중에 사용할 것입니다.

* 참조 사항

우리가 여기서 사기를 일으키는 카드를 찾는 것이 아니라 그런 사용자를 찾는 것이란 것을 명심하세요.

위의 TransactionDT를 설명 드리면 거래를 하려는데 카드가 정지되거나 카드를 누가 불법으로 썼는데 카드 주인이 모르고 넘어 간다거나 등 등에서 모두 날짜가 중요하겠죠. 

이에 대한 reference 포인트가 될 항목입니다.

In [ ]:
start_date = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

In [ ]:
print (start_date)

### Filling missing values in `card` columns

Сard columns are important but missing values.

In [ ]:
for col in train.columns:
    if 'card' in col:
        print(f"{col} has {train[col].isnull().sum()} missing values.")

'card1'의 각 값에 대한 'card2'의 가장 일반적인 값을 살펴 보겠습니다.

In [ ]:
group_with_mode = train.groupby(['card1']).agg({'card2': ['nunique', pd.Series.mode]})
group_with_mode.head(10)

'card1'의 유니크한 값이 얼마나 있는지, 1개를 초과하는 유니크한 값을 가진 'card1'의 유니크한 값이 얼마나 있는지 보겠습니다

In [ ]:
non_one = group_with_mode[group_with_mode['card2']['nunique'] > 1].shape[0]
card1_nunique = train['card1'].nunique()
print(f'Number of unique values of card1: {card1_nunique}')
print(f'Number of unique values of card1 which have more than one unique value: {non_one}')

대부분의 경우 card1의 각 값에 대해 card2 열에는 고유 한 값이 하나만 있습니다. 이것을 참조하여 누락 된 값을 채워 봅시다

In [ ]:
for card in ['card2','card3','card4','card5','card6']:
    group_with_mode = train.groupby(['card1']).agg({card: ['nunique', pd.Series.mode]})
    to_merge = group_with_mode[group_with_mode[card]['nunique'] == 1][card]['mode']
    merged = pd.merge(train['card1'], to_merge, on='card1', how='left')
    merged['mode'] = merged['mode'].fillna(train[card])
    train[card] = merged['mode']
    
    group_with_mode = test.groupby(['card1']).agg({card: ['nunique', pd.Series.mode]})
    to_merge = group_with_mode[group_with_mode[card]['nunique'] == 1][card]['mode']
    merged = pd.merge(test['card1'], to_merge, on='card1', how='left')
    merged['mode'] = merged['mode'].fillna(test[card])
    test[card] = merged['mode']

## Target and Users

라벨링 로직에 대한 정보는 https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203#588953 에서 볼 수 있습니다.
```
라벨링의 논리는 카드에 보고된 지불 거절을 사기 거래 (isFraud = 1)로 정의하고 사용자 계정, 이메일 주소 또는 이러한 속성에 직접 연결된 청구 주소를 사기로 사후에 거래하는 것입니다. 

120 일이 지난 후에도 위에 나온 것들에서 보고된 것이 없는 경우 합법적인 거래로 정의합니다 (isFraud = 0).
```

결과적으로 사기의 경우 단일 거래가 사기로 간주될 뿐만 아니라 사용자의 다른 거래도 사기로 간주된다는 것을 알 수 있기 때문에 이것은 매우 중요합니다. (다른 카드 모두 정지될 테니까요)

하지만 우리에게는 데이터에 사용자 ID가 있는 열이 없습니다. 익명 처리되었습니다. 

결과적으로 다른 열을 기반으로 사용자 ID 열을 만들어야 겠지요. 

예를 들어 주소와 카드 정보의 조합을 ID로 사용할 수 있습니다.

우리는 'D1'이 사용자의 첫 거래 이후 일수를 의미한다는 것을 알고 있습니다.

더 나아가기 전에 캐글의 컬럼 정보 부분을 다시 보겠습니다.

* TransactionDT: 주어진 포인트로부터 타임델타 (실 타임 스템프는 아님)
* TransactionAMT: 미국 달러로 결제한 거래 액수
* ProductCD: 상품 코드, 거래한 상품
* card1 — card6: 지불 수단 카드 정보, 카드 타입, 카드 분류, 은행, 국가 등
* addr: 주소
* dist: 거리
* P_ and (R__) emaildomain: 구매자의 이메일 정보
* C1-C14: 카운팅 얼마나 많은 주소가 연결되었느냐 등인데 실제 내용은 마스킹 되어 있어서 파악이 안 됨
* D1-D15: 타임델타, 예를 들어 그 전 거래부터 얼마나 되었는지 등
* M1-M9: 매치...카드와 이름 주소의 매치 등
* Vxxx: Vesta 제작 항목 랭킹, 카운팅, 기타 관련
* identity table의 항목은 네트워크 연결 정보 예를 들어 IP, ISP, Proxy, 등 또는 디지털 시그내쳐 UA/browser/os/version 등으로 거래 관련 정보를 나타낸다

DaysFromStart라는 항목을 만듭니다. 

TransactionDT에서 60초 60분 24시간을 나누어서 버림(floor)로 해서 1을 뺀 날짜를 구하고 

D1-DaysFromStart 항목을 만들어 D1-DaysFromStart 날짜를 봅니다

In [ ]:
for df in [train,test]:
    df['DaysFromStart'] = np.floor(df['TransactionDT']/(60*60*24)) - 1
    df['D1-DaysFromStart'] = df['D1'] - df['DaysFromStart']

이어서 연속으로 정보를 합쳐서 uid를 생성합니다

In [ ]:
for df in [train,test]:
    df['uid'] = df['ProductCD'].astype(str) + '_' + df['card1'].astype(str) + '_' + df['card2'].astype(str)
    df['uid'] = df['uid'] + '_' + df['card3'].astype(str) + '_' + df['card4'].astype(str)
    df['uid'] = df['uid'] + '_' + df['card5'].astype(str) + '_' + df['card6'].astype(str)
    df['uid'] = df['uid'] + '_' + df['addr1'].astype(str) + '_' + df['D1-DaysFromStart'].astype(str)

uid의 크기와 유니크한 값을 보겠습니다

In [ ]:
len(set(train['uid']).intersection(set(test['uid']))), train.uid.nunique(), test.uid.nunique()

In [ ]:
train['uid'].value_counts(dropna=False, normalize=True).head(20)

트레인 데이터와 테스트 데이터간에 일부 uid 만 겹치는 것을 볼 수 있습니다. 즉, 이 항목을 직접 적용하여 사용할 수 없으며 이를 기반으로 다양한 항목을 만들어 사용해야 할 것 같습니다.

### processing some device information and ids

DeviceInfo의 밸류 카운트를 보겠습니다

In [ ]:
train['DeviceInfo'].value_counts(dropna=False, normalize=True).head(20)

보시면, 거의 80%가 빈 값이고. 윈도우가 8%가량, iOS가 3.34% 등 나오고 삼성도 좀 보입니다.

유니크한 값이 몇개 인지 보겠습니다.

In [ ]:
train['DeviceInfo'].nunique()

아주 많은 디바이스가 있습니다. 그리고 80% 가량은 빈값이니 어느 장비인 줄 알 수가 없습니다.

이를 좀 고쳐서 항목을 만들어 보려 하겠습니다

In [ ]:
for df in [train, test]:
    df['DeviceInfo'] = df['DeviceInfo'].fillna('unknown_device').str.lower()
    df['DeviceInfo_device'] = df['DeviceInfo'].apply(lambda x: ''.join([i for i in x if i.isalpha()]))
    df['DeviceInfo_version'] = df['DeviceInfo'].apply(lambda x: ''.join([i for i in x if i.isnumeric()]))

join은 데이터를 합쳐 주는데 dtype이 같아야만 합니다

모델명 숫자 등을 지우고 종류를 정리해 보았습니다

아래에서 얼마나 줄어들었는지 봅니다

In [ ]:
df['DeviceInfo_device'].value_counts(dropna=False, normalize=True).head(20)

In [ ]:
df['DeviceInfo_version'].value_counts(dropna=False, normalize=True).head(20)

'id_30' 의 밸류 카운트를 비율로 봅니다

In [ ]:
df['id_30'].value_counts(dropna=False, normalize=True).head(20)

유니크한 값이 몇개나 있는지 보겠습니다.

In [ ]:
train['id_30'].nunique()

'id_31' 의 밸류 카운트를 비율로 봅니다

In [ ]:
df['id_31'].value_counts(dropna=False, normalize=True).head(20)

이들을 위에 DeviceInfo에 했던 것 처럼 해봅니다.

In [ ]:
for df in [train, test]:
    df['id_30'] = df['id_30'].fillna('unknown_device').str.lower()
    df['id_30_device'] = df['id_30'].apply(lambda x: ''.join([i for i in x if i.isalpha()]))
    df['id_30_version'] = df['id_30'].apply(lambda x: ''.join([i for i in x if i.isnumeric()]))

    df['id_31'] = df['id_31'].fillna('unknown_device').str.lower()
    df['id_31_device'] = df['id_31'].apply(lambda x: ''.join([i for i in x if i.isalpha()]))

빈 값에는 unknown device라 입력을 하고 합치고 줄여 봅니다

In [ ]:
df['id_30_device'].value_counts(dropna=False, normalize=True).head(20)

### process some D-columns

우리는 Card 열들은 카드 종류나 카드사 같은 정보라는 것을 위에서 보았으며

id는 사용된 OS나 디바이스 등임을 알았습니다.

이제 D항목들은 어떤 것인지 보겠습니다

일단 D1 에서 D15까지 어떤 값들이 있나 봅니다

In [ ]:
train.D1.value_counts()

In [ ]:
train.D2.value_counts()

In [ ]:
train.D3.value_counts()

In [ ]:
train.D4.value_counts()

In [ ]:
train.D5.value_counts()

In [ ]:
train.D6.value_counts()

In [ ]:
train.D7.value_counts()

In [ ]:
train.D8.value_counts()

In [ ]:
train.D9.value_counts()

In [ ]:
train.D10.value_counts()

In [ ]:
train.D11.value_counts()

In [ ]:
train.D12.value_counts()

In [ ]:
train.D13.value_counts()

In [ ]:
train.D14.value_counts()

In [ ]:
train.D15.value_counts()

D8과 D9의 분포부터 보겠습니다

In [ ]:
plt.hist(train['D8']);
plt.title('Distribution of D8 variable');

In [ ]:
plt.hist(train['D9']);
plt.title('Distribution of D9 variable');

In [ ]:
for df in [train, test]:
    # 클립 메소드를 사용하면 음수를 0으로 만들 수 있습니다
    for col in ['D'+str(i) for i in range(1,16)]:
        df[col] = df[col].clip(0)
    
    # D9을 빈 값이 아닌 것을 따로 떼어내어 D9_not_na 항목을 새로 만듭니다.
    df['D9_not_na'] = np.where(df['D9'].isna(),0,1)
    # D8을 1보다 크거나 같은 것만 떼어내어 D8_not_same_day 항목을 만듭니다
    df['D8_not_same_day'] = np.where(df['D8']>=1,1,0)
    # D8에서 소수점 이하를 뽑아서 D8_D9_decimal_dist로 정의한 후 여기서 D9을 뺍니다
    df['D8_D9_decimal_dist'] = df['D8'].fillna(0)-df['D8'].fillna(0).astype(int)
    df['D8_D9_decimal_dist'] = ((df['D8_D9_decimal_dist']-df['D9'])**2)**0.5
    df['D8'] = df['D8'].fillna(-1).astype(int)

In [ ]:
df.D9_not_na.value_counts()

In [ ]:
df.D8_not_same_day.value_counts()

In [ ]:
df.D8_D9_decimal_dist.value_counts()

In [ ]:
df.D8.value_counts()

## Process `TransactionAmt`

이제는 거래 액수에 대해 보겠습니다 

분포를 보겠습니다

In [ ]:
plt.hist(train['TransactionAmt']);
plt.title('Distribution of TransactionAmt variable');

상위 1%(퍼센타일 99) 를 봅니다 

In [ ]:
np.percentile(train['TransactionAmt'], 99)

In [ ]:
# 클립하여 0부터 5000사이의 값으로 봅니다
train['TransactionAmt'] = train['TransactionAmt'].clip(0,5000)
test['TransactionAmt']  = test['TransactionAmt'].clip(0,5000)

# 거래 액수가 일반적인지 아닌지를 봅니다
train['TransactionAmt_check'] = np.where(train['TransactionAmt'].isin(test['TransactionAmt']), 1, 0)
test['TransactionAmt_check']  = np.where(test['TransactionAmt'].isin(train['TransactionAmt']), 1, 0)

In [ ]:
train.TransactionAmt_check.value_counts()

In [ ]:
test.TransactionAmt_check.value_counts()

트레인 거래액수에서 테스트 거래 액수에 없는 것이 16,920건 테스트 거래 액수에서 트레인 거래 액수에 없는 것이 2363건인 것을 볼 수 있습니다.

다른 유용한 항목들을 만들어 봅니다.

In [ ]:
for df in [train, test]:
    df['ProductCD_card1'] = df['ProductCD'].astype(str) + '_' + df['card1'].astype(str)
    df['card1_addr1'] = df['card1'].astype(str) + '_' + df['addr1'].astype(str)
    df['TransactionAmt_dist2'] = df['TransactionAmt'].astype(str) + '_' + df['dist2'].astype(str)
    df['card3_card5'] = df['card3'].astype(str) + '_' + df['card5'].astype(str)
    df['ProductCD_TransactionAmt'] = df['ProductCD'].astype(str) + '_' + df['TransactionAmt'].astype(str)
    df['cents'] = np.round(df['TransactionAmt'] - np.floor(df['TransactionAmt']), 3)
    df['ProductCD_cents'] = df['ProductCD'].astype(str) + '_' + df['cents'].astype(str)
    df['TransactionAmt'] = np.log1p(df['TransactionAmt'])

In [ ]:
df.ProductCD_card1.value_counts()

In [ ]:
df.card1_addr1.value_counts()

In [ ]:
df.TransactionAmt_dist2.value_counts()

In [ ]:
df.ProductCD_TransactionAmt.value_counts()

In [ ]:
df.TransactionAmt.value_counts()

In [ ]:
test['ProductCD_card1']

In [ ]:
# 아래 항목들은 다른 것을 만드는데만 유용하니 이제 없애도록 하겠습니다
train = train.drop(['DaysFromStart','D1-DaysFromStart'], axis=1)
test = test.drop(['DaysFromStart','D1-DaysFromStart'], axis=1)

이제 날짜 기반 항목들을 만들어 보겠습니다.

In [ ]:
for df in [train, test]:
    # 집계를위한 임시 항목들
    df['DT'] = df['TransactionDT'].apply(lambda x: (start_date + datetime.timedelta(seconds=x)))
    df['DT_M'] = ((df['DT'].dt.year - 2017) * 12 + df['DT'].dt.month).astype(np.int8)
    df['DT_W'] = ((df['DT'].dt.year - 2017) * 52 + df['DT'].dt.weekofyear).astype(np.int8)
    df['DT_D'] = ((df['DT'].dt.year - 2017) * 365 + df['DT'].dt.dayofyear).astype(np.int16)

    df['DT_hour'] = df['DT'].dt.hour.astype(np.int8)
    df['DT_day_week'] = df['DT'].dt.dayofweek.astype(np.int8)
    df['DT_day_month'] = df['DT'].dt.day.astype(np.int8)

    # 잠재적 솔로 항목
    df['is_december'] = df['DT'].dt.month
    df['is_december'] = (df['is_december'] == 12).astype(np.int8)

## Validation

올바른 유효성 검사는 매우 중요합니다. 

로컬 점수가 리더 보드 점수와 관련이 있는 경우 모델을 로컬에서 학습 및 비교하고 때때로 제출할 수 있습니다. 

로컬 점수가 리더 보드 점수와 관련이 없는 경우 모델이 좋은지 여부를 실제로 추정 할 수 없습니다.

데이터에 날짜가 있으므로 일종의 시계열 검증을 사용해야합니다. 

즉, 훈련 데이터가 항상 검증 데이터보다 시간 상으로 먼저 있어야합니다.

이 커널에서는 학습을 위해 처음 80 % 데이터를, 유효성 검사를 위해 마지막 20%를 취하겠습니다. 

신뢰성을 보장하기 위해 다양한 랜덤 시드로 여러 모델을 훈련시키고 점수를 평균화합니다.

In [ ]:
y = train['isFraud']
X = train.drop(['isFraud'], axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

### Processing the data separately

검증이 올바른지 확인하기 위해 트레인 및 검증 데이터를 트레인 및 테스트 데이터를 다룰 때와 동일하게 처리해야합니다.

이를 위해 두 개의 데이터 프레임을 처리하는 함수를 작성합니다.

이 함수에서 모델링에 사용되지 않을 항목을 위해 remove_features 목록을 만들 것입니다.

그들 중 일부는 일시적인 것이었고 일부는 쓸모 없거나 나쁜 것으로 판명되었습니다.

먼저 프리퀀시 인코딩을 위한 함수를 작성해 보겠습니다.

두 개의 데이터 프레임을 사용하고 이러한 데이터 프레임에서 하나의 열을 결합하고 그 안에있는 각 범주의 수를 계산합니다.

카테고리 대신 이러한 값을 사용하여 새 열을 만드는 데 사용됩니다.

이것은 널리 사용되는 kaggle 기술입니다.



In [ ]:
def freq_encode_full(df1, df2, col, normalize=True):
    """
    Encode

    https://www.kaggle.com/cdeotte/high-scoring-lgbm-malware-0-702-0-775
    """
    df = pd.concat([df1[col], df2[col]])
    freq_dict = df.value_counts(dropna=False, normalize=normalize).to_dict()
    col_name = col + '_freq_enc_full'
    return col_name, freq_dict

In [ ]:
def process_data(df_train: pd.DataFrame, df_test: pd.DataFrame):
    # 제자리에서 변경되지 않도록 복사하십시오.
    train_df = df_train.copy()
    test_df = df_test.copy()
    remove_features = ['TransactionID', 'TransactionDT', 'DT', 'DT_M', 'DT_W', 'DT_D', 'DT_hour', 'DT_day_week',
                       'DT_day_month',
                       'ProductCD_card1', 'card1_addr1', 'TransactionAmt_dist2', 'card3_card5', 'uid',
                       'D5_DT_W_std_score',
                       'ProductCD_TransactionAmt_DT_W', 'D4_DT_D_std_score', 'D15_DT_D_std_score', 'D3_DT_W_std_score',
                       'D11_DT_W_std_score',
                       'card3_card5_DT_W_week_day_dist', 'card5_DT_W_week_day_dist', 'D10_DT_D_std_score',
                       'card3_card5_DT_D', 'ProductCD_cents_DT_D',
                       'D4_DT_W_std_score', 'D15_DT_W_std_score', 'uid_DT_D', 'card3_DT_W_week_day_dist',
                       'D10_DT_W_std_score', 'D8_DT_D_std_score',
                       'card3_card5_DT_W', 'ProductCD_cents_DT_W', 'uid_DT_W', 'D8_DT_W_std_score',
                       'ProductCD_TransactionAmt']

    """
     다음 코드는 다음을 수행합니다.
     * 학습 및 테스트 데이터에서 특정 열의 값을 결합하고 'value_counts'계산
     *`valid_card`를 하나 이상의 값이 있는 카테고리로 정의
     * 트레인 데이터의 열 범주도 테스트 데이터에 있으면 유지하고, 그렇지 않으면 None으로 바꿉니다.
     * 트레인 데이터의 열 범주가`valid_card`에 있으면 유지하고, 그렇지 않으면 None으로 바꿉니다.
     * 테스트 데이터에 대해 동일하게 수행
    """
    for col in ['card1', 'ProductCD_card1', 'card1_addr1', 'TransactionAmt_dist2']:
        valid_card = pd.concat([train_df[[col]], test_df[[col]]])
        valid_card = valid_card[col].value_counts()

        valid_card = valid_card[valid_card > 2]
        valid_card = list(valid_card.index)

        train_df[col] = np.where(train_df[col].isin(test_df[col]), train_df[col], np.nan)
        train_df[col] = np.where(train_df[col].isin(valid_card), train_df[col], np.nan)

        test_df[col] = np.where(test_df[col].isin(valid_card), test_df[col], np.nan)
        test_df[col] = np.where(test_df[col].isin(train_df[col]), test_df[col], np.nan)

    # 이전과 같이 값이 하나의 데이터 세트에만있는 경우 None으로 바꿉니다
    for col in ['card2', 'card3', 'card4', 'card5', 'card6']:
        train_df[col] = np.where(train_df[col].isin(test_df[col]), train_df[col], np.nan)
        test_df[col] = np.where(test_df[col].isin(train_df[col]), test_df[col], np.nan)

    ####### 투레인 데이터에서 지난달의 최대 값으로 C 열을 자릅니다.
    i_cols = ['C' + str(i) for i in range(1, 15)]
    for df in [train_df, test_df]:
        for col in i_cols:
            max_value = train_df[train_df['DT_M'] == train_df['DT_M'].max()][col].max()
            df[col] = df[col].clip(None, max_value)

    ####### V feature - NaN group agg
    # null 값과 열 목록을 포함하는 dictionary가 됩니다.
    nans_groups = {}
    nans_df = pd.concat([train_df, test_df]).isna()

    i_cols = ['V' + str(i) for i in range(1, 340)]
    for col in i_cols:
        # NaN 값을 세워 봅니다
        cur_group = nans_df[col].sum()
        if cur_group > 0:
            try:
                nans_groups[cur_group].append(col)
            except:
                nans_groups[cur_group] = [col]

    for i, (n_group, n_cols) in enumerate(nans_groups.items()):
        for df in [train_df, test_df]:
            df[f'nan_group_{i}_sum'] = df[n_cols].sum(axis=1)
            df[f'nan_group_{i}_mean'] = df[n_cols].mean(axis=1)
            df[f'nan_group_{i}_std'] = df[n_cols].std(axis=1)

    del nans_groups, nans_df
    remove_features += i_cols
    # 너무나 많은 공간을 차지합니다. 필요한 것만 남기고 drop 시킵니다.
    i_cols = [i for i in i_cols if i not in ['V258', 'V306', 'V307', 'V308', 'V294']]
    train_df = train_df.drop(i_cols, axis=1)
    test_df = test_df.drop(i_cols, axis=1)

    # frequency encoding. 인코딩한 항목과 오리지날 항목을 `remove_features`에 더합니다
    i_cols = [
        'ProductCD_TransactionAmt', 'ProductCD_cents', 'cents',
        'DeviceInfo', 'DeviceInfo_device', 'DeviceInfo_version',
        'id_30', 'id_30_device', 'id_30_version',
        'id_31', 'id_31_device',
        'id_33',
    ]
    for col in i_cols:
        col_name, freq_dict = freq_encode_full(train_df, test_df, col)

        train_df[col_name] = train_df[col].map(freq_dict).astype('float32')
        test_df[col_name] = test_df[col].map(freq_dict).astype('float32')

        remove_features.append(col)

    # 원래 항목을 유지하면서 frequency encoding을 합니다. 
    i_cols = ['id_01', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_13',
              'id_14', 'id_17', 'id_18', 'id_19', 'id_20',
              'id_21', 'id_22', 'id_24', 'id_25', 'id_26', 'card1', 'card2', 'card3', 'card5', 'ProductCD_card1',
              'card1_addr1', 'TransactionAmt_dist2']
    for col in i_cols:
        col_name, freq_dict = freq_encode_full(train_df, test_df, col)

        train_df[col_name] = train_df[col].map(freq_dict).astype('float32')
        test_df[col_name] = test_df[col].map(freq_dict).astype('float32')

    return train_df, test_df, remove_features

In [ ]:
%%time
X_train, X_val, remove_features = process_data(X_train, X_val)

* 참조; %%time은 CPU times와 wall time을 출력합니다 %%time은 셀 전체에 대해, %time은 첫 줄에 대한 것만을 출력합니다.

## Feature engineering

이미 피쳐엔지니어링을 좀 했지만, 기본적인 것 부터 시작하겠습니다.

범주 항목에 label encoding부터 하겠습니다

In [ ]:
def feature_engineering(df_train: pd.DataFrame, df_test: pd.DataFrame, remove_features: list):
    train_df = df_train.copy()
    test_df = df_test.copy()

    # Label Encoding    
    for f in train_df.columns:
        if train_df[f].dtype == 'object' or test_df[f].dtype == 'object':
            train_df[f] = train_df[f].fillna('unseen_before_label')
            test_df[f] = test_df[f].fillna('unseen_before_label')
            lbl = LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            train_df[f] = train_df[f].astype('category')
            test_df[f] = test_df[f].astype('category')

    print('remove_features:', remove_features)

    feature_columns = [col for col in list(train_df) if col not in remove_features]
    categorical_features = [col for col in feature_columns if train_df[col].dtype.name == 'category']
    categorical_features = [col for col in categorical_features if col not in remove_features]

    print(f'train.shape : {train_df[feature_columns].shape}, test.shape : {test_df[feature_columns].shape}')

    return train_df[feature_columns], test_df[feature_columns], categorical_features


In [ ]:
X_train, X_val, categorical_features = feature_engineering(X_train, X_val, remove_features)

# Training the model

모델을 트레인하고 우리의 접근법을 검증할 함수를 만들어 봅니다.

요즘 테이블형 데이터는 그래디언트 부스팅 모델로 많이 하는데 우리도 여기서 LightGBM을 사용해 보겠습니다.

In [ ]:
lgb_params = {
    'objective': 'binary',
    'metric': 'None',
    'learning_rate': 0.01,
    'num_leaves': 2**8,
    'max_bin': 255,
    'max_depth': -1,
    'bagging_freq': 5,
    'bagging_fraction': 0.7,
    'bagging_seed': seed,
    'feature_fraction': 0.7,
    'feature_fraction_seed': seed,
    'first_metric_only': True,
    'verbose': 100,
    'n_jobs': -1,
    'seed': seed,
}

여기서 competition metric은 roc_auc으로 표현됩니다. 

기본적인 LightGBM으로는 너무 느릴 수가 잇어서 여기서 우리 나름의 맞춤 평가 함수를 만들어 봅니다.

In [ ]:
@jit
def fast_auc(y_true, y_prob):
    """
    fast roc_auc computation: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc


def eval_auc(y_pred, y_true):
    """
    Fast auc eval function for lgb.
    """
    return 'auc', fast_auc(y_true.get_label(), y_pred), True


In [ ]:
def make_val_prediction(X_train, y_train, X_val, y_val, seed=0, seed_range=3, lgb_params=None,
                        category_cols=None):
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)

    auc_scores = []
    best_iterations = []
    val_preds = np.zeros((X_val.shape[0], 3))

    feature_importance_df = pd.DataFrame()
    feature_importance_df['feature'] = X_train.columns.tolist()
    feature_importance_df['gain_importance'] = 0

    for i, s in enumerate(range(seed, seed + seed_range)):
        seed_everything(s)
        params = lgb_params.copy()
        params['seed'] = s
        params['bagging_seed'] = s
        params['feature_fraction_seed'] = s

        clf = lgb.train(params, train_data, 10000, valid_sets=[train_data, val_data],
                        categorical_feature=categorical_features,
                        early_stopping_rounds=500, feval=eval_auc, verbose_eval=200)

        best_iteration = clf.best_iteration
        best_iterations.append(best_iteration)
        val_pred = clf.predict(X_val, best_iteration)
        val_preds[:, i] = val_pred

        auc = fast_auc(y_val, val_pred)
        auc_scores.append(auc)
        print('seed:', s, ', auc:', auc, ', best_iteration:', best_iteration)

        feature_importance_df['gain_importance'] += clf.feature_importance('gain') / seed_range

    auc_scores = np.array(auc_scores)
    best_iterations = np.array(best_iterations)
    best_iteration = int(np.mean(best_iterations))

    avg_pred_auc = fast_auc(y_val, np.mean(val_preds, axis=1))
    print(
        f'avg pred auc: {avg_pred_auc:.5f}, avg auc: {np.mean(auc_scores):.5f}+/-{np.std(auc_scores):.5f}, avg best iteration: {best_iteration}')

    feature_importance_df = feature_importance_df.sort_values(by='gain_importance', ascending=False).reset_index(
        drop=True)
    plt.figure(figsize=(16, 12));
    sns.barplot(x="gain_importance", y="feature", data=feature_importance_df[:50])
    plt.title('LGB Features (avg over folds)');

    return feature_importance_df, best_iteration, val_preds

In [ ]:
feature_importance_df, best_iteration, val_preds = make_val_prediction(X_train, y_train, X_val, y_val, category_cols=categorical_features,
                                                       lgb_params=lgb_params)

## Making prediction

이제 우리는 예측을 만들어 냅니다.

StratifiedFold에 대한 전체 데이터에 대한 모델을 학습 할 것입니다.
모델에서 유효성 검사를 사용하지 않지만 과적합을 피하기 위해 고정된 수의 반복을 설정합니다.
훈련은 폴드에서 수행됩니다.

In [ ]:
def prediction(X, y, X_test, best_iteration, seed=seed, category_cols=None, n_folds=5):
    print('best iteration:', best_iteration)
    preds = np.zeros((X_test.shape[0], n_folds))

    print(X.shape, X_test.shape)

    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)

    for i, (trn_idx, _) in enumerate(skf.split(X, y)):
        fold = i + 1

        tr_x, tr_y = X.iloc[trn_idx, :], y.iloc[trn_idx]

        tr_data = lgb.Dataset(tr_x, label=tr_y)

        clf = lgb.train(lgb_params, tr_data, best_iteration, categorical_feature=category_cols)
        preds[:, i] = clf.predict(X_test)

    return preds


In [ ]:
%%time
X_full, X_test, remove_features = process_data(X, test.copy())
X_full, X_test, categorical_features = feature_engineering(X_full, X_test, remove_features)

In [ ]:
preds = prediction(X_full, y, X_test, best_iteration, category_cols=categorical_features)

In [ ]:
sub['isFraud'] = np.mean(preds, axis=1)
sub.to_csv('sub_1.csv', index=False)

## More feature engineering

이제는 D열을 위주로 작업을 해보겠습니다

일정 기간 (주 또는 일)이 포함된 함수를 정의하고 이 기간을 기준으로 값을 그룹화한 다음 정규화된 값을 최소 / 최대 및 평균 / 표준으로 계산하는 것으로 만들어 봅니다.

In [ ]:
def values_normalization(dt_df, periods, columns):
    for period in periods:
        for col in columns:
            new_col = col + '_' + period

            dt_df[col] = dt_df[col].astype(float)

            temp_min = dt_df.groupby([period])[col].agg(['min']).reset_index()
            temp_min.index = temp_min[period].values
            temp_min = temp_min['min'].to_dict()

            temp_max = dt_df.groupby([period])[col].agg(['max']).reset_index()
            temp_max.index = temp_max[period].values
            temp_max = temp_max['max'].to_dict()

            temp_mean = dt_df.groupby([period])[col].agg(['mean']).reset_index()
            temp_mean.index = temp_mean[period].values
            temp_mean = temp_mean['mean'].to_dict()

            temp_std = dt_df.groupby([period])[col].agg(['std']).reset_index()
            temp_std.index = temp_std[period].values
            temp_std = temp_std['std'].to_dict()

            dt_df['temp_min'] = dt_df[period].map(temp_min)
            dt_df['temp_max'] = dt_df[period].map(temp_max)
            dt_df['temp_mean'] = dt_df[period].map(temp_mean)
            dt_df['temp_std'] = dt_df[period].map(temp_std)

            dt_df[new_col + '_min_max'] = (dt_df[col] - dt_df['temp_min']) / (dt_df['temp_max'] - dt_df['temp_min'])
            dt_df[new_col + '_std_score'] = (dt_df[col] - dt_df['temp_mean']) / (dt_df['temp_std'])
            del dt_df['temp_min'], dt_df['temp_max'], dt_df['temp_mean'], dt_df['temp_std']
    return dt_df

In [ ]:
def feature_engineering(df_train: pd.DataFrame, df_test: pd.DataFrame, remove_features: list):
    # 변경되지 않도록 복사합니다
    train_df = df_train.copy()
    test_df = df_test.copy()

    i_cols = ['D' + str(i) for i in range(1, 16)]

    ####### Values Normalization
    i_cols.remove('D1')
    i_cols.remove('D2')
    i_cols.remove('D9')
    periods = ['DT_D', 'DT_W']
    for df in [train_df, test_df]:
        df = values_normalization(df, periods, i_cols)

    # TransactionAmt Normalization
    periods = ['DT_D', 'DT_W']
    for df in [train_df, test_df]:
        df = values_normalization(df, periods, ['TransactionAmt'])

    # normalize by max train value
    for col in ['D1', 'D2']:
        for df in [train_df, test_df]:
            df[col + '_scaled'] = df[col] / train_df[col].max()

    # frequency encoding
    i_cols = ['D' + str(i) for i in range(1, 16)] + ['DT_D', 'DT_W', 'ProductCD_TransactionAmt', 'ProductCD_cents']
    for col in i_cols:
        col_name, freq_dict = freq_encode_full(train_df, test_df, col)

        train_df[col_name] = train_df[col].map(freq_dict).astype('float32')
        test_df[col_name] = test_df[col].map(freq_dict).astype('float32')

        remove_features.append(col)

    # Label Encoding    
    for f in train_df.columns:
        if train_df[f].dtype == 'object' or test_df[f].dtype == 'object':
            train_df[f] = train_df[f].fillna('unseen_before_label')
            test_df[f] = test_df[f].fillna('unseen_before_label')
            lbl = LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            train_df[f] = train_df[f].astype('category')
            test_df[f] = test_df[f].astype('category')

    print('remove_features:', remove_features)
    print(f'train.shape : {train_df.shape}, test.shape : {test_df.shape}')

    ########################### Final features list
    feature_columns = [col for col in list(train_df) if col not in remove_features]
    print('feature_columns:', len(feature_columns))
    categorical_features = [col for col in feature_columns if train_df[col].dtype.name == 'category']
    categorical_features = [col for col in categorical_features if col not in remove_features]

    return train_df[feature_columns], test_df[feature_columns], categorical_features

In [ ]:
%%time
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_val, remove_features = process_data(X_train, X_val)
X_train, X_val, categorical_features = feature_engineering(X_train, X_val, remove_features)

In [ ]:
feature_importance_df, best_iteration, val_preds = make_val_prediction(X_train, y_train, X_val, y_val, category_cols=categorical_features,
                                                       lgb_params=lgb_params)

In [ ]:
%%time
X_full, X_test, remove_features = process_data(X, test.copy())
X_full, X_test, categorical_features = feature_engineering(X_full, X_test, remove_features)
preds = prediction(X_full, y, X_test, best_iteration, category_cols=categorical_features)
sub['isFraud'] = np.mean(preds, axis=1)
sub.to_csv('sub_2.csv', index=False)

## More feature engineering 2

이제는 C-columns에 대한 피쳐 엔지니어링을 좀 해 보겠습니다

In [ ]:
def feature_engineering(df_train: pd.DataFrame, df_test: pd.DataFrame, remove_features: list):
    # 원본이 변경되지 않도록 복사해 사용합니다
    train_df = df_train.copy()
    test_df = df_test.copy()

    i_cols = ['D' + str(i) for i in range(1, 16)]

    ####### Values Normalization
    i_cols.remove('D1')
    i_cols.remove('D2')
    i_cols.remove('D9')
    periods = ['DT_D', 'DT_W']
    for df in [train_df, test_df]:
        df = values_normalization(df, periods, i_cols)

    # TransactionAmt Normalization
    periods = ['DT_D', 'DT_W']
    for df in [train_df, test_df]:
        df = values_normalization(df, periods, ['TransactionAmt'])

    # normalize by max train value
    for col in ['D1', 'D2']:
        for df in [train_df, test_df]:
            df[col + '_scaled'] = df[col] / train_df[col].max()

    # frequency encoding
    i_cols = ['D' + str(i) for i in range(1, 16)] + ['DT_D', 'DT_W', 'ProductCD_TransactionAmt', 'ProductCD_cents']
    for col in i_cols:
        col_name, freq_dict = freq_encode_full(train_df, test_df, col)

        train_df[col_name] = train_df[col].map(freq_dict).astype('float32')
        test_df[col_name] = test_df[col].map(freq_dict).astype('float32')

        remove_features.append(col)

    i_cols = ['C' + str(i) for i in range(1, 15)]
    # 여기서는 C column의 0들을 카운트합니다
    for df in [train_df, test_df]:
        df['c_cols_0_bin'] = ''
        for c in i_cols:
            df['c_cols_0_bin'] += (df[c] == 0).astype(int).astype(str)
    col_name, freq_dict = freq_encode_full(train_df, test_df, 'c_cols_0_bin')

    train_df[col_name] = train_df['c_cols_0_bin'].map(freq_dict).astype('float32')
    test_df[col_name] = test_df['c_cols_0_bin'].map(freq_dict).astype('float32')

    for col in i_cols:
        col_name, freq_dict = freq_encode_full(train_df, test_df, col)

        train_df[col_name] = train_df[col].map(freq_dict).astype('float32')
        test_df[col_name] = test_df[col].map(freq_dict).astype('float32')

    # Label Encoding    
    for f in train_df.columns:
        if train_df[f].dtype == 'object' or test_df[f].dtype == 'object':
            train_df[f] = train_df[f].fillna('unseen_before_label')
            test_df[f] = test_df[f].fillna('unseen_before_label')
            lbl = LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            train_df[f] = train_df[f].astype('category')
            test_df[f] = test_df[f].astype('category')

    print('remove_features:', remove_features)
    print(f'train.shape : {train_df.shape}, test.shape : {test_df.shape}')

    ########################### Final features list
    feature_columns = [col for col in list(train_df) if col not in remove_features]
    print('feature_columns:', len(feature_columns))
    categorical_features = [col for col in feature_columns if train_df[col].dtype.name == 'category']
    categorical_features = [col for col in categorical_features if col not in remove_features]

    return train_df[feature_columns], test_df[feature_columns], categorical_features

In [ ]:
%%time
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_val, remove_features = process_data(X_train, X_val)
X_train, X_val, categorical_features = feature_engineering(X_train, X_val, remove_features)

In [ ]:
feature_importance_df, best_iteration, val_preds = make_val_prediction(X_train, y_train, X_val, y_val, category_cols=categorical_features,
                                                       lgb_params=lgb_params)

In [ ]:
%%time
X_full, X_test, remove_features = process_data(X, test.copy())
X_full, X_test, categorical_features = feature_engineering(X_full, X_test, remove_features)
preds = prediction(X_full, y, X_test, best_iteration, category_cols=categorical_features)
sub['isFraud'] = np.mean(preds, axis=1)
sub.to_csv('sub_3.csv', index=False)

## More feature engineering - aggregations

데이터에 대한 다양한 집계를 계산합니다.

In [ ]:
def uid_aggregation(train_df, test_df, main_columns, uids, aggregations):
    for main_column in main_columns:
        for col in uids:
            for agg_type in aggregations:
                new_col_name = col + '_' + main_column + '_' + agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col, main_column]]])
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                    columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name] = test_df[col].map(temp_df)
    return train_df, test_df

In [ ]:
def feature_engineering(df_train: pd.DataFrame, df_test: pd.DataFrame, remove_features: list):
    # 복사해 둡니다
    train_df = df_train.copy()
    test_df = df_test.copy()

    i_cols = ['D' + str(i) for i in range(1, 16)]

    ####### Values Normalization
    i_cols.remove('D1')
    i_cols.remove('D2')
    i_cols.remove('D9')
    periods = ['DT_D', 'DT_W']
    for df in [train_df, test_df]:
        df = values_normalization(df, periods, i_cols)

    # TransactionAmt Normalization
    periods = ['DT_D', 'DT_W']
    for df in [train_df, test_df]:
        df = values_normalization(df, periods, ['TransactionAmt'])

    for col in ['D1', 'D2']:
        for df in [train_df, test_df]:
            df[col + '_scaled'] = df[col] / train_df[col].max()

    i_cols = ['D' + str(i) for i in range(1, 16)] + ['DT_D', 'DT_W', 'ProductCD_TransactionAmt', 'ProductCD_cents']
    for col in i_cols:
        col_name, freq_dict = freq_encode_full(train_df, test_df, col)

        train_df[col_name] = train_df[col].map(freq_dict).astype('float32')
        test_df[col_name] = test_df[col].map(freq_dict).astype('float32')

        remove_features.append(col)

    i_cols = ['C' + str(i) for i in range(1, 15)]

    for df in [train_df, test_df]:
        df['c_cols_0_bin'] = ''
        for c in i_cols:
            df['c_cols_0_bin'] += (df[c] == 0).astype(int).astype(str)
    col_name, freq_dict = freq_encode_full(train_df, test_df, 'c_cols_0_bin')

    train_df[col_name] = train_df['c_cols_0_bin'].map(freq_dict).astype('float32')
    test_df[col_name] = test_df['c_cols_0_bin'].map(freq_dict).astype('float32')

    for col in i_cols:
        col_name, freq_dict = freq_encode_full(train_df, test_df, col)

        train_df[col_name] = train_df[col].map(freq_dict).astype('float32')
        test_df[col_name] = test_df[col].map(freq_dict).astype('float32')

    i_cols = ['TransactionAmt', 'id_01', 'id_02', 'id_05', 'id_06', 'id_09', 'id_14', 'dist1'] + ['C' + str(i) for i in
                                                                                                  range(1, 15)]
    uids = ['card1', 'card2', 'card3', 'card5', 'uid', 'card3_card5']
    aggregations = ['mean', 'std']

    # uIDs aggregations
    train_df, test_df = uid_aggregation(train_df, test_df, i_cols, uids, aggregations)

    i_cols = [
                 'V258',
                 'V306', 'V307', 'V308', 'V294'
             ] + ['D' + str(i) for i in range(1, 16)]
    uids = ['uid', 'card3_card5']
    aggregations = ['mean', 'std']
    train_df, test_df = uid_aggregation(train_df, test_df, i_cols, uids, aggregations)

    # Label Encoding    
    for f in train_df.columns:
        if train_df[f].dtype == 'object' or test_df[f].dtype == 'object':
            train_df[f] = train_df[f].fillna('unseen_before_label')
            test_df[f] = test_df[f].fillna('unseen_before_label')
            lbl = LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            train_df[f] = train_df[f].astype('category')
            test_df[f] = test_df[f].astype('category')

    print('remove_features:', remove_features)
    print(f'train.shape : {train_df.shape}, test.shape : {test_df.shape}')

    ########################### Final features list
    feature_columns = [col for col in list(train_df) if col not in remove_features]
    print('feature_columns:', len(feature_columns))
    categorical_features = [col for col in feature_columns if train_df[col].dtype.name == 'category']
    categorical_features = [col for col in categorical_features if col not in remove_features]

    return train_df[feature_columns], test_df[feature_columns], categorical_features

In [ ]:
%%time
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_val, remove_features = process_data(X_train, X_val)
X_train, X_val, categorical_features = feature_engineering(X_train, X_val, remove_features)

In [ ]:
feature_importance_df, best_iteration, val_preds = make_val_prediction(X_train, y_train, X_val, y_val, category_cols=categorical_features,
                                                       lgb_params=lgb_params)

In [ ]:
%%time
X_full, X_test, remove_features = process_data(X, test.copy())
X_full, X_test, categorical_features = feature_engineering(X_full, X_test, remove_features)
preds = prediction(X_full, y, X_test, best_iteration, category_cols=categorical_features)
sub['isFraud'] = np.mean(preds, axis=1)
sub.to_csv('sub_4.csv', index=False)

## More feature engineering - time-block frequency encoding

이것은 일반적인 frequency 인코딩과 유사하지만 해당 기간 내에 값을 인코딩합니다.

In [ ]:
def timeblock_frequency_encoding(train_df, test_df, periods, columns):
    for period in periods:
        for col in columns:
            new_col = col + '_' + period
            print('timeblock frequency encoding:', new_col)
            train_df[new_col] = train_df[col].astype(str) + '_' + train_df[period].astype(str)
            test_df[new_col] = test_df[col].astype(str) + '_' + test_df[period].astype(str)

            temp_df = pd.concat([train_df[[new_col]], test_df[[new_col]]])
            fq_encode = temp_df[new_col].value_counts(normalize=True).to_dict()

            train_df[new_col] = train_df[new_col].map(fq_encode)
            test_df[new_col] = test_df[new_col].map(fq_encode)

            train_df[new_col] = train_df[new_col] / train_df[period + '_freq_enc_full']
            test_df[new_col] = test_df[new_col] / test_df[period + '_freq_enc_full']

    return train_df, test_df

In [ ]:
def feature_engineering(df_train: pd.DataFrame, df_test: pd.DataFrame, remove_features: list):
    # 복사해 둡니다
    train_df = df_train.copy()
    test_df = df_test.copy()

    i_cols = ['D' + str(i) for i in range(1, 16)]

    ####### Values Normalization
    i_cols.remove('D1')
    i_cols.remove('D2')
    i_cols.remove('D9')
    periods = ['DT_D', 'DT_W']
    for df in [train_df, test_df]:
        df = values_normalization(df, periods, i_cols)

    # TransactionAmt Normalization
    periods = ['DT_D', 'DT_W']
    for df in [train_df, test_df]:
        df = values_normalization(df, periods, ['TransactionAmt'])

    for col in ['D1', 'D2']:
        for df in [train_df, test_df]:
            df[col + '_scaled'] = df[col] / train_df[col].max()

    i_cols = ['D' + str(i) for i in range(1, 16)] + ['DT_D', 'DT_W', 'ProductCD_TransactionAmt', 'ProductCD_cents']
    for col in i_cols:
        col_name, freq_dict = freq_encode_full(train_df, test_df, col)

        train_df[col_name] = train_df[col].map(freq_dict).astype('float32')
        test_df[col_name] = test_df[col].map(freq_dict).astype('float32')

        remove_features.append(col)

    i_cols = ['C' + str(i) for i in range(1, 15)]

    for df in [train_df, test_df]:
        df['c_cols_0_bin'] = ''
        for c in i_cols:
            df['c_cols_0_bin'] += (df[c] == 0).astype(int).astype(str)
    col_name, freq_dict = freq_encode_full(train_df, test_df, 'c_cols_0_bin')

    train_df[col_name] = train_df['c_cols_0_bin'].map(freq_dict).astype('float32')
    test_df[col_name] = test_df['c_cols_0_bin'].map(freq_dict).astype('float32')

    for col in i_cols:
        col_name, freq_dict = freq_encode_full(train_df, test_df, col)

        train_df[col_name] = train_df[col].map(freq_dict).astype('float32')
        test_df[col_name] = test_df[col].map(freq_dict).astype('float32')

    i_cols = ['TransactionAmt', 'id_01', 'id_02', 'id_05', 'id_06', 'id_09', 'id_14', 'dist1'] + ['C' + str(i) for i in
                                                                                                  range(1, 15)]
    uids = ['card1', 'card2', 'card3', 'card5', 'uid', 'card3_card5']
    aggregations = ['mean', 'std']

    # uIDs aggregations
    train_df, test_df = uid_aggregation(train_df, test_df, i_cols, uids, aggregations)

    i_cols = [
                 'V258',
                 'V306', 'V307', 'V308', 'V294'
             ] + ['D' + str(i) for i in range(1, 16)]
    uids = ['uid', 'card3_card5']
    aggregations = ['mean', 'std']
    train_df, test_df = uid_aggregation(train_df, test_df, i_cols, uids, aggregations)

    i_cols = ['ProductCD_TransactionAmt', 'ProductCD_cents', 'ProductCD_cents', 'uid', 'card3_card5']
    periods = ['DT_D', 'DT_W']
    train_df, test_df = timeblock_frequency_encoding(train_df, test_df, periods, i_cols)

    # Label Encoding    
    for f in train_df.columns:
        if train_df[f].dtype == 'object' or test_df[f].dtype == 'object':
            train_df[f] = train_df[f].fillna('unseen_before_label')
            test_df[f] = test_df[f].fillna('unseen_before_label')
            lbl = LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            train_df[f] = train_df[f].astype('category')
            test_df[f] = test_df[f].astype('category')

    print('remove_features:', remove_features)
    print(f'train.shape : {train_df.shape}, test.shape : {test_df.shape}')

    ########################### Final features list
    feature_columns = [col for col in list(train_df) if col not in remove_features]
    print('feature_columns:', len(feature_columns))
    categorical_features = [col for col in feature_columns if train_df[col].dtype.name == 'category']
    categorical_features = [col for col in categorical_features if col not in remove_features]

    return train_df[feature_columns], test_df[feature_columns], categorical_features


In [ ]:
%%time
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_val, remove_features = process_data(X_train, X_val)
X_train, X_val, categorical_features = feature_engineering(X_train, X_val, remove_features)

In [ ]:
feature_importance_df, best_iteration, val_preds = make_val_prediction(X_train, y_train, X_val, y_val, category_cols=categorical_features,
                                                       lgb_params=lgb_params)

In [ ]:
%%time
X_full, X_test, remove_features = process_data(X, test.copy())
X_full, X_test, categorical_features = feature_engineering(X_full, X_test, remove_features)
preds = prediction(X_full, y, X_test, best_iteration, category_cols=categorical_features)
sub['isFraud'] = np.mean(preds, axis=1)
sub.to_csv('sub_5.csv', index=False)